# Fudan PRML22 Spring Final Project

*Your name and Student ID: 陈朦伊, 19307110382*

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet, and a .pdf report file) with your assignment submission.*

**Congratulations, you have come to the last challenge!**

Having finished the past two assignments, we think all you gugs already have a solid foundation in the field of machine learning and deep learning. And now you are qualified to apply machine learning algorithms to the real-world tasks you are interested in, or start your machine learning research. 

**In this final project, you are free to choose a topic you are passionate about. The project can be an application one, a theoretical one or implementing your own amazing machine learning/deep learning framework like a toy pytorch. If you don't have any idea, we will also provide you with a default one you can play with.** 

**! Notice: If you want to work on your own idea, you have to email the TA (lip21[at]m.fudan.edu.cn) to give a simple project proposal first before May 22, 2022.** 

## Default Project: Natural Language Inference

![Sherlock](./img/inference.jpg)

The default final project this semester is a NLP task called "Natural Language Inference". Though deep neural networks have demonstrated astonishing performance in many tasks like text classification and generation, you might somehow think they are just "advanced statistics" but far from *intelligent* machines. One intelligent machine must be able to reason, you may think. And in this default final project, your aim is to design a machine which can conduct inference. The machine can know that "A man inspects the uniform of a figure in some East Asian country" is contradictory to "The man is sleeping", and "a soccer game with multiple males playing." entails "some men are playing a sport".

The dataset we use this time is the Original Chinese Natural Language Inference (OCNLI) dataset[1]. It is a chinese NLI dataset with about 50k training data and 3k development data. The sentence pairs in the dataset are labeled as "entailment", "neutral" and "contradiction". Due to they release the test data without its labels, we select 5k data pairs from the training data as labeled test data, and the other 45k data as your t. You can visit the [GitHub link](https://github.com/CLUEbenchmark/OCNLI) for more information.

After you finished the NLI task with the full 50k training set, you have to complete an advanced challenge. You have to select **at most 5k data** from the training set as labeled training set, leaving the other training data as unlabeled training set, then use these labeled and unlabeled data to finish the same NLI task. You can randomly choosing the 5k training data but can also think up some ideas to select more **important data** as labeled training data. Like assignment1, you may have to think how to use the unlabeled training data.

You can use the deep learning frameworks like paddle, pytorch, tensorflow in your experiment but not more high-level libraries like Huggingface. Please write down the version of them in the './requirements.txt' file.

**! Notice: You CAN NOT use any other people's pretrained model like 'bert-base-chinese' in this default project. You are encouraged to design your own model and algorithm, no matter it looks naive or not.**

NLI is a traditional but promising NLP task, and you can search the Google/Bing for more information. Some key words can be "natural language inference with attention", "training data selection", "semi-surpervised learning", "unsupervised representation learning" and so on.

## 1. Setup

import the libraries and load the dataset here.

In [1]:
# setup code
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import Vectors
from model.util import load_iters
from model.models import ESIM
from tqdm import tqdm

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%load_ext autoreload
%autoreload 2

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = './dataset'

train_data_file = dataset_path + '/train.json'
dev_data_file = dataset_path + '/dev.json'
test_data_file = dataset_path + '/test.json'

In [3]:
def read_ocnli_file(data_file):
    # read the ocnli file. feel free to change it. 
    print ("loading data from ", data_file)
    
    text_outputs = []
    label_outputs = []
    
    label_to_idx = {"entailment": 0, "neutral": 1, "contradiction": 2}
    
    with open(data_file, 'r', encoding="utf-8") as f:
        line = f.readline()
        while line:
            line = json.loads(line.strip())
            text_a, text_b, label = line['sentence1'], line['sentence2'],line['label']
            label_id = label_to_idx[label.strip()]
            
            text_outputs.append((text_a,text_b))
            label_outputs.append(label_id)

            line = f.readline()
                
    print ("there are ", len(label_outputs), "sentence pairs in this file.")
    return text_outputs, label_outputs


training_data, training_labels = read_ocnli_file(train_data_file)
dev_data, dev_labels = read_ocnli_file(dev_data_file)
test_data, test_labels = read_ocnli_file(test_data_file)

loading data from  ./dataset/train.json
there are  45437 sentence pairs in this file.
loading data from  ./dataset/dev.json
there are  2950 sentence pairs in this file.
loading data from  ./dataset/test.json
there are  5000 sentence pairs in this file.


In [20]:
print ("training data samples: ", training_data[:5])
print ("training labels samples: ", training_labels[:5])

training data samples:  [('对,对,对,对,对,具体的答复.', '要的是抽象的答复'), ('当前国际形势仍处于复杂而深刻的变动之中', '一个月后将发生世界战争'), ('在全县率先推行宅基地有偿使用,全乡20年无须再扩大宅基地', '宅基地有偿使用获得较好成果,将在更大范围实施。'), ('上海马路上的喧声也是老调子', '上海有很多条马路'), ('那你看看第二封信什么时候到吧.', '第一封信已经收到了。')]
training labels samples:  [2, 1, 1, 1, 1]


## 2. Exploratory Data Analysis (5 points)

Your may have to explore the dataset and do some analysis first.

In [21]:
print("Training size: ",len(training_data))
print("Dev size: ",len(dev_data))
print("Test size: ", len(test_data))


Training size:  45437
Dev size:  2950
Test size:  5000


## 3. Methodology (50 points)

Build and evaluate your model here.

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def show_example(premise, hypothesis, label, TEXT, LABEL):
    tqdm.write('Label: ' + LABEL.vocab.itos[label])
    tqdm.write('premise: ' + ' '.join([TEXT.vocab.itos[i] for i in premise]))
    tqdm.write('hypothesis: ' + ' '.join([TEXT.vocab.itos[i] for i in hypothesis]))


def eval(data_iter, name, epoch=None, use_cache=False):
    if use_cache:
        model.load_state_dict(torch.load('best_model.ckpt'))
    model.eval()
    correct_num = 0
    err_num = 0
    total_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_iter):
            premise, premise_lens = batch.premise
            hypothesis, hypothesis_lens = batch.hypothesis
            labels = batch.label

            output = model(premise, premise_lens, hypothesis, hypothesis_lens)
            predicts = output.argmax(-1).reshape(-1)
            loss = loss_func(output, labels)
            total_loss += loss.item()
            correct_num += (predicts == labels).sum().item()
            err_num += (predicts != batch.label).sum().item()

    acc = correct_num / (correct_num + err_num)
    if epoch is not None:
        tqdm.write(
            "Epoch: %d, %s Acc: %.3f, Loss %.3f" % (epoch + 1, name, acc, total_loss))
    else:
        tqdm.write(
            "%s Acc: %.3f, Loss %.3f" % (name, acc, total_loss))
    return acc


def train(train_iter, dev_iter, loss_func, optimizer, epochs, patience=5, clip=5):
    best_acc = -1
    patience_counter = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_iter):
            premise, premise_lens = batch.premise
            hypothesis, hypothesis_lens = batch.hypothesis
            labels = batch.label
            # show_example(premise[0],hypothesis[0], labels[0], TEXT, LABEL)

            model.zero_grad()
            output = model(premise, premise_lens, hypothesis, hypothesis_lens)
            loss = loss_func(output, labels)
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
        tqdm.write("Epoch: %d, Train Loss: %d" % (epoch + 1, total_loss))

        acc = eval(dev_iter, "Dev", epoch)
        if acc<best_acc:
            patience_counter +=1
        else:
            best_acc = acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.ckpt')
        if patience_counter >= patience:
            tqdm.write("Early stopping: patience limit reached, stopping...")
            break

In [4]:
data_path = 'dataset'
vectors = None
freeze = False

BATCH_SIZE = 32
HIDDEN_SIZE = 600  # every LSTM's(forward and backward) hidden size is half of HIDDEN_SIZE
EPOCHS = 20
DROPOUT_RATE = 0.5
LAYER_NUM = 1
LEARNING_RATE = 4e-4
PATIENCE = 5
CLIP = 10
EMBEDDING_SIZE = 50

train_iter, dev_iter, test_iter, TEXT, LABEL, _ = load_iters(BATCH_SIZE, device, data_path, vectors)

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torc

In [13]:
model = ESIM(len(TEXT.vocab), len(LABEL.vocab.stoi),
                 EMBEDDING_SIZE, HIDDEN_SIZE, DROPOUT_RATE, LAYER_NUM,
                 TEXT.vocab.vectors, freeze).to(device)
print(f'The model has {count_parameters(model):,} trainable parameters')
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_func = nn.CrossEntropyLoss()
train(train_iter, dev_iter, loss_func, optimizer, EPOCHS,PATIENCE, CLIP)

The model has 8,561,203 trainable parameters


100%|██████████| 1420/1420 [00:36<00:00, 39.32it/s]


Epoch: 1, Train Loss: 1560
Epoch: 1, Dev Acc: 0.374, Loss 101.965


100%|██████████| 1420/1420 [00:35<00:00, 39.89it/s]


Epoch: 2, Train Loss: 1560
Epoch: 2, Dev Acc: 0.305, Loss 103.415


...

Epoch: 6, Train Loss: 81
Epoch: 6, Dev Acc: 0.323, Loss 423.738


100%|██████████| 1420/1420 [00:36<00:00, 38.95it/s]


Epoch: 7, Train Loss: 37
Epoch: 7, Dev Acc: 0.323, Loss 483.419


100%|██████████| 1420/1420 [00:36<00:00, 39.40it/s]


Epoch: 8, Train Loss: 27
Epoch: 8, Dev Acc: 0.323, Loss 477.412
Early stopping: patience limit reached, stopping...


In [14]:
eval(test_iter, "Test", use_cache=True)

Test Acc: 0.340, Loss 180.162


0.34

## 4. Attention Visualization (10 points)

Visualize the attention matrix in your model here.

## 5. Model Attack (30 points)

Attack your model here.

## 6. Conclusion (5 points)

Write down your conclusion here.

本次pj让我们大致了解了NLI任务的工作流程与基本实现方法，然而由于期末季时间太紧，没能实现第二个挑战，十分抱歉。

## Reference

[1] OCNLI: Original Chinese Natural Language Inference, arxiv: https://arxiv.org/abs/2010.05444